In [47]:
import pandas as pd
import numpy as np
import jinja2

In [ ]:
#import du ficher csv référentiel géographique
ref_geo = pd.read_csv("Data/Données-immo/fr-esr-referentiel-geographique.csv", encoding='latin1', sep= ';')
ref_geo.head().style.format()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_226776\1419685486.py:2: DtypeWarning: Columns (5,6,7,12,17) have mixed types. Specify dtype option on import or set low_memory=False.
  ref_geo = pd.read_csv("ressources/Données-immo/fr-esr-referentiel-geographique.csv", encoding='latin1', sep= ';')


,regrgp_nom,reg_nom,reg_nom_old,aca_nom,dep_nom,com_code,com_code1,com_code2,com_id,com_nom_maj_court,com_nom_maj,com_nom,uu_code,uu_id,uucr_id,uucr_nom,ze_id,dep_code,dep_id,dep_nom_num,dep_num_nom,aca_code,aca_id,reg_code,reg_id,reg_code_old,reg_id_old,fd_id,fr_id,fe_id,uu_id_99,au_code,au_id,auc_id,auc_nom,uu_id_10,geolocalisation
0,Province,Auvergne-Rhône-Alpes,Rhône-Alpes,Lyon,Ain,1001,1001,1001,C01001,L ABERGEMENT CLEMENCIAT,L'ABERGEMENT-CLEMENCIAT,L'Abergement-Clémenciat,nan,SO,CR01001,L'Abergement-Clémenciat,ZE8213,1,D001,Ain (01),01 - Ain,10,A10,84,R84,82,R82,FD111,FR11,FE1,SO,nan,AU997,C01001,L'Abergement-Clémenciat,SO,"46.1534255214,4.92611354223"
1,Province,Auvergne-Rhône-Alpes,Rhône-Alpes,Lyon,Ain,1002,1002,1002,C01002,L ABERGEMENT DE VAREY,L'ABERGEMENT-DE-VAREY,L'Abergement-de-Varey,nan,SO,CR01002,L'Abergement-de-Varey,ZE8201,1,D001,Ain (01),01 - Ain,10,A10,84,R84,82,R82,FD111,FR11,FE1,SO,2,AU002,AU002,Lyon,SO,"46.0091878776,5.42801696363"
2,Province,Auvergne-Rhône-Alpes,Rhône-Alpes,Lyon,Ain,1003,1003,1003,C01003,AMAREINS,AMAREINS,Amareins,nan,SO,SO,Amareins,SO,1,D001,Ain (01),01 - Ain,10,A10,84,R84,82,R82,FD111,FR11,FE1,SO,SO,SO,SO,SO,SO,nan
3,Province,Auvergne-Rhône-Alpes,Rhône-Alpes,Lyon,Ain,1004,1004,1004,C01004,AMBERIEU EN BUGEY,AMBERIEU-EN-BUGEY,Ambérieu-en-Bugey,1303.000000,UU01303,UU01303,Ambérieu-en-Bugey,ZE8201,1,D001,Ain (01),01 - Ain,10,A10,84,R84,82,R82,FD111,FR11,FE1,UU01303,2,AU002,AU002,Lyon,UU01302,"45.9608475114,5.3729257777"
4,Province,Auvergne-Rhône-Alpes,Rhône-Alpes,Lyon,Ain,1005,1005,1005,C01005,AMBERIEUX EN DOMBES,AMBERIEUX-EN-DOMBES,Ambérieux-en-Dombes,nan,SO,CR01005,Ambérieux-en-Dombes,ZE8213,1,D001,Ain (01),01 - Ain,10,A10,84,R84,82,R82,FD111,FR11,FE1,SO,2,AU002,AU002,Lyon,SO,"45.9961799872,4.91227250796"


In [49]:
#création de la dataframe departement destinée à la DB et export en csv
departement = pd.DataFrame()
departement[['code_dep','nom_dep','code_reg','nom_reg']] = ref_geo[['dep_code','dep_nom','reg_code','reg_nom']]
departement = departement.set_index(['code_dep'], drop=True)
#suprression des doublons car autant de lignes que de communes
departement = departement.drop_duplicates()
departement.head()

,nom_dep,code_reg,nom_reg
code_dep,,,
1,Ain,84,Auvergne-Rhône-Alpes
2,Aisne,32,Hauts-de-France
3,Allier,84,Auvergne-Rhône-Alpes
4,Alpes-de-Haute-Provence,93,Provence-Alpes-Côte d'Azur
5,Hautes-Alpes,93,Provence-Alpes-Côte d'Azur


In [50]:
# creation de la db region et suppression des colonnes region de la table departement
region = pd.DataFrame()
region[['code_reg','nom_reg']] = departement[['code_reg','nom_reg']]
region = region.set_index(['code_reg'], drop=True)
#suprression des doublons car autant de lignes que de communes
region = region.drop_duplicates()
region.to_csv('Data/region.csv', sep=  ';')
region.head()


,nom_reg
code_reg,
84,Auvergne-Rhône-Alpes
32,Hauts-de-France
93,Provence-Alpes-Côte d'Azur
44,Grand Est
76,Occitanie


In [51]:
# Suppression du nom de region de la table departement et enregistrement en csv
departement = departement.drop('nom_reg', axis = 1)
departement.to_csv('Data/departement.csv', sep=  ';')
departement.head()

,nom_dep,code_reg
code_dep,,
1,Ain,84
2,Aisne,32
3,Allier,84
4,Alpes-de-Haute-Provence,93
5,Hautes-Alpes,93


In [52]:
#Import des données utiles pour communes
commune2 = ref_geo[['com_nom','com_code','dep_code','com_id']]
commune2 = commune2.rename({'com_code':'code_dep_code_com'}, axis = 1)
commune2 = commune2.set_index('code_dep_code_com')
commune2.index = commune2.index.astype(str)
commune2.head()

,com_nom,dep_code,com_id
code_dep_code_com,,,
1001,L'Abergement-Clémenciat,1,C01001
1002,L'Abergement-de-Varey,1,C01002
1003,Amareins,1,C01003
1004,Ambérieu-en-Bugey,1,C01004
1005,Ambérieux-en-Dombes,1,C01005


In [ ]:
# création de la dataframe données_communes à partir du fichier donnees_communes.csv
donnees_communes = pd.read_csv("Data/Données-immo/donnees_communes.csv", sep= ';', encoding='latin-1')
donnees_communes.head()
donnees_communes.shape[0]

34991

In [54]:
# Création de la table commmune destinée à la DB avec les données utiles
commune = pd.DataFrame()
commune[['code_com','code_dep','population']] = donnees_communes[['CODCOM','CODDEP',"PTOT"]].reindex()
# création de l'index code_dep_code_com
commune['code_dep'] = commune['code_dep'].str.lstrip("0")
commune['code_dep_code_com'] = commune['code_dep']+commune['code_com'].astype(str).str.zfill(3)
commune = commune.set_index('code_dep_code_com')
commune.index = commune.index.astype(str)
commune.head()

,code_com,code_dep,population
code_dep_code_com,,,
1001,1,1,798
1002,2,1,257
1004,4,1,14514
1005,5,1,1776
1006,6,1,118


In [55]:
#Utilisation du fichier référentiel géographique pour insérer les noms de commune (problèmes d'encodage dans le fichier communes)
commune = pd.merge(commune2['com_nom'], commune,  on='code_dep_code_com')
commune = commune.rename({'com_nom':'nom_commune'}, axis = 1)
commune.to_csv('data/commune.csv', sep = ";")
commune.head()

,nom_commune,code_com,code_dep,population
code_dep_code_com,,,,
1001,L'Abergement-Clémenciat,1,1,798
1002,L'Abergement-de-Varey,2,1,257
1004,Ambérieu-en-Bugey,4,1,14514
1005,Ambérieux-en-Dombes,5,1,1776
1006,Ambléon,6,1,118


In [ ]:
#import du ficher csv référentiel géographique
valeurs_foncieres = pd.read_csv("Data/Données-immo/Valeurs-foncières.csv", encoding='latin1', sep= ';', low_memory = False)
valeurs_foncieres.head().style.format()

,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Code type de voie,Type de voie,Code voie,Voie,Code ID commune,Code postal,Commune,Code departement,Code commune,Préfixe de section,Section,No plan,No Volume,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,Nom de l'acquereur
0,nan,nan,nan,nan,nan,nan,nan,1,2020/01/02,Vente,165000,347.000000,nan,0,RUE,20,DU CHATEAU,1,1170.000000,CHEVRY,01,103,nan,A,302,nan,12,"48,22",nan,nan,nan,nan,nan,nan,nan,nan,2,2,Appartement,nan,48,3,nan,nan,nan,GUIRAO
1,nan,nan,nan,nan,nan,nan,nan,1,2020/01/02,Vente,355680,4.000000,nan,15,BD,1000,EDOUARD BAUDOIN,205,6160.000000,ANTIBES,06,4,nan,CP,186,nan,132,"39,11",nan,nan,nan,nan,nan,nan,nan,nan,2,2,Appartement,nan,40,1,nan,nan,nan,HARNOIS
2,nan,nan,nan,nan,nan,nan,nan,1,2020/01/02,Vente,229500,20.000000,B,0,RUE,3975,MARCEAU,142,6000.000000,NICE,06,88,nan,LS,169,nan,99,"80,25",nan,nan,nan,nan,nan,nan,nan,nan,1,2,Appartement,nan,82,3,nan,nan,nan,ROGIER
3,nan,nan,nan,nan,nan,nan,nan,1,2020/01/02,Vente,125000,550.000000,nan,3,RTE,1011,DES VESPINS RN7,228,6700.000000,SAINT LAURENT DU VAR,06,123,nan,AO,348,nan,242,"27,51",nan,nan,nan,nan,nan,nan,nan,nan,1,2,Appartement,nan,27,1,nan,nan,nan,BOCQUIER
4,nan,nan,nan,nan,nan,nan,nan,1,2020/01/02,Vente,90000,9300.000000,nan,18,RES,A084,LES ARPEGES BD DES ABA,326,13400.000000,AUBAGNE,13,5,nan,AW,224,nan,218,"47,33",nan,nan,nan,nan,nan,nan,nan,nan,1,2,Appartement,nan,47,2,nan,nan,nan,GUILLOSSOU


In [57]:
valeurs_foncieres['Valeur fonciere'] = valeurs_foncieres['Valeur fonciere'].str.replace(',','.').astype(float)

In [58]:
valeurs_foncieres.iloc[:,[27,44]].dtypes

Surface Carrez du 1er lot     object
Surface terrain              float64
dtype: object

In [59]:
# création de la df vente (temporaire) destinée à la DB à partir de la df valeurs_foncieres avec les noms de colonnes voulus
vente = pd.DataFrame()
vente[['prix','date',
    'no_voie', 'BTQ', 'type_voie', 'voie', 'commune',
    'code_dep','code_com','type_local',
    'surface_carrez','surface_bati','nb_pieces'
    ]] = valeurs_foncieres[['Valeur fonciere', 'Date mutation',
        'No voie','B/T/Q','Type de voie', 'Voie', 'Commune',
        'Code departement', 'Code commune', 'Type local',
        'Surface Carrez du 1er lot',
        'Surface reelle bati', 'Nombre pieces principales']] \
    .reindex()
# calcul et ajout de la clé secondaire code_dep_code_com
vente['code_dep'] = vente['code_dep'].str.lstrip("0")
vente.insert(7, 'code_dep_code_com', vente['code_dep']+vente['code_com'].astype(str).str.zfill(3))
vente = vente.drop(['code_dep','code_com'], axis = 1)
# assignation d'un num à chaque vente = index de la df
vente.index.name = 'id_vente'
#correction des types de données et gestion des NA
vente['no_voie'] = vente['no_voie'].fillna("0").astype(int)
vente['surface_carrez'] = vente['surface_carrez'].str.replace(',','.').astype(float)
#passage en minuscule
vente['voie'] = vente['voie'].str.lower()
vente['commune'] = vente['commune'].str.lower()
vente.head()

,prix,date,no_voie,BTQ,type_voie,voie,commune,code_dep_code_com,type_local,surface_carrez,surface_bati,nb_pieces
id_vente,,,,,,,,,,,,
0,165000.0,2020/01/02,347,NaN,RUE,du chateau,chevry,1103,Appartement,48.22,48,3
1,355680.0,2020/01/02,4,NaN,BD,edouard baudoin,antibes,6004,Appartement,39.11,40,1
2,229500.0,2020/01/02,20,B,RUE,marceau,nice,6088,Appartement,80.25,82,3
3,125000.0,2020/01/02,550,NaN,RTE,des vespins rn7,saint laurent du var,6123,Appartement,27.51,27,1
4,90000.0,2020/01/02,9300,NaN,RES,les arpeges bd des aba,aubagne,13005,Appartement,47.33,47,2


In [60]:
# Séparation de la table vente temporaire en tables vente et bien
bien = vente
# Suppression des données en trop dans la table vente et ajout de la référence id_bien 
vente['id_bien'] = vente.index
vente = vente.drop(['no_voie','BTQ','type_voie','voie','commune','code_dep_code_com','type_local','surface_carrez','surface_bati','nb_pieces'], axis=1)
vente.to_csv('Data/vente.csv', sep=  ';')
vente.head()

,prix,date,id_bien
id_vente,,,
0,165000.0,2020/01/02,0
1,355680.0,2020/01/02,1
2,229500.0,2020/01/02,2
3,125000.0,2020/01/02,3
4,90000.0,2020/01/02,4


In [61]:
# Rédindexation de la table bien et suppression des lignes correspondant aux ventes
bien.index.names = ['id_bien']
bien = bien.drop(['prix','date','id_bien','commune'], axis = 1)
bien.to_csv('Data/bien.csv', sep=  ';')
bien.head()

,no_voie,BTQ,type_voie,voie,code_dep_code_com,type_local,surface_carrez,surface_bati,nb_pieces
id_bien,,,,,,,,,
0,347,NaN,RUE,du chateau,1103,Appartement,48.22,48,3
1,4,NaN,BD,edouard baudoin,6004,Appartement,39.11,40,1
2,20,B,RUE,marceau,6088,Appartement,80.25,82,3
3,550,NaN,RTE,des vespins rn7,6123,Appartement,27.51,27,1
4,9300,NaN,RES,les arpeges bd des aba,13005,Appartement,47.33,47,2


In [62]:
vente['prix'].sum()

8634982159.95